<a href="https://colab.research.google.com/github/bankvis7/cp-axtra-training-lab-3/blob/main/lab1_1_attention_transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 1 Part 1: LLM Fine-tune/Inference โดยใช้ HF Transformers

เนื้อหา:
- การ Training/Fine-tuning และ Inference ของ LLM โดยใช้ PyTorch/Transformers
- **Eager Attention** การใช้งาน attention แบบไม่มีการ optimize

โดยจะวัด:
- เวลาในการโหลดโมเดลและหน่วยความจำ
- ความเร็วและ throughput ของ inference
- ความเร็วในการ training (1 epoch บนชุดข้อมูลขนาดเล็ก)
- การใช้หน่วยความจำระหว่างการ training

**โมเดล**: Gemma 2B

---

## Hugging Face Token

ในการเข้าถึงโมเดล Gemma จะต้องยืนยันตัวตนกับ Hugging Face โดยใช้ read access token จาก [Hugging Face](https://huggingface.co/settings/tokens) และเพิ่มลงใน Colab secrets manager ให้ตั้งชื่อ secret ว่า `HF_TOKEN`

หรือ Run Cell ต่อไปนี้เพื่อ Login ผ่าน GUI

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

### การตั้งค่า Hugging Face Access Token

1. สมัครบัญชีที่ [huggingface.co](https://huggingface.co/)
2. เลือกโมเดลที่ต้องการใช้งาน ในตัวอย่างนี้ใช้ [Gemma 2B](https://huggingface.co/google/gemma-2b)
3. ยอมรับข้อตกลงการใช้งานโมเดล (สำหรับโมเดล Gemma ต้องขอสิทธิ์ใช้งานก่อน)
4. ไปที่หน้า [Hugging Face Tokens](https://huggingface.co/settings/tokens)
5. คลิก "Create new token"
6. เลือกประเภทสิทธิ์เป็น "Read"
7. คัดลอก Token และเก็บไว้ใน Colab โดยใช้ Secrets (ไอคอนรูปกุญแจที่เมนูด้านซ้าย)
8. ตั้งชื่อ (Name) เป็น `HF_TOKEN` และค่าของ Token (Value) เป็นรหัสที่คัดลอกมา จากนั้นติ๊กให้ Notebook มีสิทธิ์เข้าถึง
9. รีสตาร์ท Colab session หลังจากตั้งค่า Token เสร็จ


### ขอสิทธิ์ใช้งาน Gemma

ในการใช้งานโมเดล Gemma คุณต้องทำการขอสิทธิ์จาก Google ก่อน ดูรายละเอียดได้ที่ https://ai.google.dev/gemma/docs/setup

หากคุณยังไม่มีบัญชี Kaggle ให้สมัครที่ [kaggle.com](https://www.kaggle.com/) จากนั้นทำตามขั้นตอนต่อไปนี้:

1. ไปที่หน้า Model Card ของ Gemma บน Kaggle
2. คลิก “Request Access”
3. กรอกแบบฟอร์มและยอมรับ เงื่อนไขการใช้งาน

* * *

## T4 GPU Runtime

แลปนี้จำเป็นต้องใช้ GPU ในการประมวลผล ให้เปิดใช้งาน T4 GPU runtime โดยไปที่ "**Runtime/Connect (มุมบนขวา)**" -> "**Change runtime type**" และเลือก "**T4 GPU**" ภายใต้ "**Hardware accelerator**"

ตรวจสอบการเชื่อมต่อกับ GPU ด้วยคำสั่งต่อไปนี้:

In [ ]:
!nvidia-smi

* * *
## การติดตั้ง Library

In [ ]:
%%capture
!pip install -q transformers accelerate bitsandbytes peft datasets trl scipy

In [ ]:
import torch
import time
import gc
import numpy as np
import pandas as pd
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments
)
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer

# ตรวจสอบความพร้อมของ GPU
print(f"CUDA พร้อมใช้งาน: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    gpu_props = torch.cuda.get_device_properties(0)
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory(หน่วยความจำ): {gpu_props.total_memory / 1e9:.2f} GB")
    print(f"เวอร์ชัน CUDA: {torch.version.cuda}")

## การตั้งค่าโมเดล

In [ ]:
# การตั้งค่าโมเดล
MODEL_NAME = "google/gemma-2b-it"

# การตั้งค่าการ fine-tune
NUM_TRAIN_SAMPLES = 500
NUM_TRAIN_EPOCHS = 1
BATCH_SIZE = 2
GRADIENT_ACCUMULATION_STEPS = 4
LEARNING_RATE = 2e-4

## สร้างฟังก์ชันสำหรับวัดหน่วยความจำ

In [ ]:
def get_gpu_memory_gb():
    """ดึงข้อมูลการใช้หน่วยความจำ GPU ปัจจุบันเป็น GB"""
    if torch.cuda.is_available():
        return torch.cuda.memory_allocated() / 1e9
    return 0

def get_peak_memory_gb():
    """ดึงข้อมูลการใช้หน่วยความจำ GPU สูงสุดเป็น GB"""
    if torch.cuda.is_available():
        return torch.cuda.max_memory_allocated() / 1e9
    return 0

def clear_memory():
    """ล้างหน่วยความจำ GPU และระบบ"""
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.reset_peak_memory_stats()

def format_time(seconds):
    """จัดรูปแบบวินาทีให้อ่านได้ง่าย"""
    return f"{seconds:.2f}วินาที ({seconds/60:.2f}นาที)"

def print_section(title):
    """พิมพ์ส่วนหัวที่จัดรูปแบบแล้ว"""
    print(f"\n{'='*80}")
    print(f"{title.center(80)}")
    print(f"{'='*80}\n")

---

# ขั้นตอนที่ 1: การโหลดโมเดล

โหลด Gemma 2B ด้วย **eager attention**

In [ ]:
print_section("กำลังโหลดโมเดลด้วย Eager Attention")

clear_memory()
start_time = time.time()
mem_before = get_gpu_memory_gb()


model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    device_map="auto",
    attn_implementation="eager"
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

load_time = time.time() - start_time
mem_after = get_gpu_memory_gb()
memory_used = mem_after - mem_before

print(f"  โหลดโมเดลเรียบร้อยแล้ว")
print(f"  การทำงานของ Attention: eager")
print(f"  เวลาในการโหลด: {format_time(load_time)}")
print(f"  หน่วยความจำที่ใช้: {memory_used:.3f} GB")
print(f"  หน่วยความจำ GPU ปัจจุบัน: {mem_after:.3f} GB")

---

# ขั้นตอนที่ 2: การทดสอบ Inference

ทดสอบความเร็วของ inference ด้วย eager attention

In [ ]:
# prompts สำหรับทดสอบ inference
TEST_PROMPTS = [
    "Explain Quantum Computing in simple terms",
    "What is a Fibonacci sequence?",
    "Create a Python function to reverse string."
]

In [ ]:
print_section("การทดสอบ Inference - Eager Attention")

clear_memory()
times = []
outputs = []


inputs = tokenizer(TEST_PROMPTS[0], return_tensors="pt").to(model.device)
with torch.no_grad():
    _ = model.generate(**inputs, max_new_tokens=50, do_sample=False)

clear_memory()
mem_before = get_gpu_memory_gb()

print(f"\nกำลังรัน inference บน {len(TEST_PROMPTS)} prompts...\n")
for i, prompt in enumerate(TEST_PROMPTS, 1):
    print(f"Prompt {i}/{len(TEST_PROMPTS)}: {prompt[:50]}...")
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    start = time.time()
    with torch.no_grad():
        output = model.generate(**inputs, max_new_tokens=50, do_sample=False)
    elapsed = time.time() - start
    times.append(elapsed)

    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    outputs.append(decoded)
    print(f"  เวลา: {elapsed:.4f}วินาที")

mem_after = get_gpu_memory_gb()
peak_mem = get_peak_memory_gb()

# ผลลัพธ์
avg_time = np.mean(times)
std_time = np.std(times)
tokens_per_sec = 50 / avg_time

print(f"\n{'='*60}")
print("ผลลัพธ์ INFERENCE (Eager Attention)")
print(f"{'='*60}")
print(f"เวลาเฉลี่ยต่อ Prompt: {avg_time:.4f}วินาที (±{std_time:.4f}วินาที)")
print(f"Tokens/วินาที: {tokens_per_sec:.2f}")
print(f"หน่วยความจำสูงสุด: {peak_mem:.3f} GB")
print(f"{'='*60}")

# เก็บผลลัพธ์
inference_results = {
    'avg_time': avg_time,
    'std_time': std_time,
    'tokens_per_sec': tokens_per_sec,
    'peak_memory': peak_mem,
    'outputs': outputs
}

## ตัวอย่าง Outputs

In [ ]:
print("\nตัวอย่าง Inference Outputs:\n")
for i, (prompt, output) in enumerate(zip(TEST_PROMPTS, outputs), 1):
    print(f"{'='*80}")
    print(f"Prompt {i}: {prompt}")
    print(f"{'-'*80}")
    print(f"Output: {output[:200]}...")
    print()

---

# ขั้นตอนที่ 3: ทดสอบการ Fine-tuning

## โหลดชุดข้อมูล

เราจะทดลองขั้นตอนการ fine-tune โดยใช้ชุดข้อมูล instruction-tuned ชื่อ FineTome-100K จาก huggingface (https://huggingface.co/datasets/mlabonne/FineTome-100k)

In [ ]:
print("โหลดชุดข้อมูล")
dataset = load_dataset("mlabonne/FineTome-100k", split=f"train[:{NUM_TRAIN_SAMPLES}]")

def format_dataset(example):
    """จัดรูปแบบ conversations เป็นคู่ instruction-response"""
    # แยก conversation
    conversations = example.get("conversations", [])
    if len(conversations) < 2:
        return {"text": ""}

    # ดึง instruction ของผู้ใช้และ response ของ assistant
    user_msg = conversations[0].get("value", "")
    assistant_msg = conversations[1].get("value", "")

    # จัดรูปแบบเป็น instruction-response
    text = f"ผู้ใช้: {user_msg}\n\nผู้ช่วย: {assistant_msg}"
    return {"text": text}

dataset = dataset.map(format_dataset, remove_columns=dataset.column_names)
# กรองตัวอย่างที่ว่างออก
dataset = dataset.filter(lambda x: len(x["text"]) > 0)

print(f"โหลดชุดข้อมูลแล้ว: {len(dataset)} ตัวอย่าง")
print(f"ตัวอย่าง:\n{dataset[0]['text'][:200]}...")

Fine-tuning ด้วย LoRA เป็นเวลา 1 epoch เพื่อวัดประสิทธิภาพ training

In [ ]:
print_section("การทดสอบ fine-tune - Eager Attention")

clear_memory()

# เพิ่ม LoRA adapters
lora_config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model_lora = get_peft_model(model, lora_config)
model_lora.print_trainable_parameters()


training_args = TrainingArguments(
    output_dir="./output_baseline",
    num_train_epochs=NUM_TRAIN_EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    learning_rate=LEARNING_RATE,
    fp16=True,
    logging_steps=10,
    save_strategy="no",
    report_to="none"
)

# สร้าง trainer
trainer = SFTTrainer(
    model=model_lora,
    args=training_args,
    train_dataset=dataset,
)

print(f"\nเริ่มการ fine-tune {len(dataset)} ตัวอย่างเป็นเวลา {NUM_TRAIN_EPOCHS} epoch...\n")

mem_before = get_gpu_memory_gb()
start_time = time.time()

train_result = trainer.train()

train_time = time.time() - start_time
mem_after = get_gpu_memory_gb()
peak_mem = get_peak_memory_gb()

# ผลลัพธ์
print(f"\n{'='*60}")
print("ผลลัพธ์การ Fine-tune (Eager Attention)")
print(f"{'='*60}")
print(f"เวลาในการ Fine-tune: {format_time(train_time)}")
print(f"ตัวอย่าง/วินาที: {train_result.metrics['train_samples_per_second']:.2f}")
print(f"หน่วยความจำสูงสุด: {peak_mem:.3f} GB")
print(f"Loss สุดท้าย: {train_result.metrics['train_loss']:.4f}")
print(f"{'='*60}")

# เก็บผลลัพธ์
training_results = {
    'train_time': train_time,
    'samples_per_sec': train_result.metrics['train_samples_per_second'],
    'peak_memory': peak_mem,
    'final_loss': train_result.metrics['train_loss']
}

----